# Brownian Motion

In [ ]:
using KadanoffBaym
using LinearAlgebra
using PyPlot
using PyCall
qt = pyimport("qutip")
np = pyimport("numpy")

font_size = 16
# font_size = 20

PyPlot.matplotlib.rc("text", usetex=true)
PyPlot.matplotlib.rc("font", family="serif", size=font_size)
PyPlot.matplotlib.rc("axes", labelsize=font_size)
PyPlot.matplotlib.rc("xtick.major", size=8)
PyPlot.matplotlib.rc("ytick.major", size=8)
PyPlot.matplotlib.rc("xtick.minor", visible=true, size=4)
PyPlot.matplotlib.rc("ytick.minor", visible=true, size=4)
PyPlot.matplotlib.rc("xtick", top=true, direction="inout")
PyPlot.matplotlib.rc("ytick", right=true, direction="inout")

## KadanoffBaym.jl

In [ ]:
# F = GreenFunction(reshape(ComplexF64[1.0 + 1.0im], 1, 1), KadanoffBaym.Classical)

# f_vert(_, _, t1, t2) = [-0.5im * (10 - 1.0im * 2.0) * F[t1, t2]]
# f_diag(_, times, t)  = [-1.0im * (10 - 1.0im * 2.0) * F[t, t]]

# F_ana = (t, t′) -> F[1, 1] * exp(-1.0im * (10 - 1.0im * 2.0) * (t + t′)/2);

In [ ]:
T = 4.0
lambda = 1.
t_scale = (iszero(lambda) ? one(lambda) : abs(lambda))

N_0 = [1., 3., 5.]
Ds = [8., 4., 1.]

Fs = [zeros(Float64, 1) for _ in 1:length(N_0)];
F_anas = [zeros(Float64, 1) for _ in 1:length(N_0)];
solts = [zeros(Float64, 1) for _ in 1:length(N_0)];

function solution!(idx::Int64, N_0::Float64, D::Float64)
    
    F = GreenFunction(reshape(Float64[N_0], 1, 1), KadanoffBaym.Classical)

    f_vert(_, _, t1, t2) = [-lambda * F[t1, t2]]
    f_diag(_, times, t)  = [-2lambda * F[t, t] + D] # (println(times[t]); [-2lambda * F[t, t] + D])

    F_ana = (t, t′) -> (F[1, 1] - D/(2 * lambda)) * exp(-lambda * (t + t′)) + D/(2 * lambda) * exp(-lambda * abs(t - t′))
    
    sol = kbsolve(f_vert, f_diag, [F], (0.0, T), dtini=1e-15, atol=1e-9, rtol=1e-7)
    
    n = length(sol.t) - 1
    
    Fs[idx] = [F.data[k, k] for k in 1:n+1]
    F_anas[idx] = F_ana.(sol.t, sol.t);
    solts[idx] = sol.t
    return F
end;

In [ ]:
map(i -> solution!(i, N_0[i], Ds[i]), 1:length(N_0))

## Plotting

In [ ]:
cmap = "gist_heat";
colors = ["C0", "C1", "C2"];
lss = ["-", "--", "-."];

In [ ]:
figure(figsize=(7, 3))

ax = subplot(121)
for (k, F) in enumerate(Fs)
    plot(solts[k], F, "-", label="\$xxx\$", lw=1.5, c=colors[k], ls=lss[k])
#     plot(solts[k], F_anas[k], ls="--", lw=4.0, alpha=0.4, c=colors[k])
end
xlim(0, lambda * T)
ylim(0, 1.1 * Ds[1]/(2lambda))
yticks([0, 1, 2, 3, 4, 5])
xlabel("\$\\lambda t\$")
ylabel("\$F(t, t)\$")

ax = subplot(122)
plot([], [], label="\$D/\\lambda\$", c="w")
for (k, F) in enumerate(Fs)
    semilogy(solts[k], abs.(F  .- F_anas[k]), lw=1.5, c=colors[k], label="\$"*string(Ds[k]/(lambda))*"\$", ls=lss[k])
end

xlim(0, lambda * T)
ylim(1e-9, 1e-6)
# yticks([0, 1, 2, 3, 4, 5])
xlabel("\$\\lambda t\$")
ylabel("\$\\left|F(t, t) - \\mathcal{F}(t, t)\\right|\$", labelpad=16)
ax.yaxis.set_label_position("right")
ax.legend(loc="best", handlelength=1.9, frameon=false, borderpad=0, labelspacing=0.25)
# ticklabel_format(axis="y", style="sci", scilimits=(-0, 0))

tight_layout(pad=0.25, w_pad=1, h_pad=0)

savefig("brownian_motion_example_T.pdf")

In [ ]:
Y, X = np.meshgrid(solts[1][1:end], solts[1][1:end])
F = solution!(1, N_0[1], Ds[1]);
F_ana = (t, t′) -> (F[1, 1] - Ds[1]/(2 * lambda)) * exp(-lambda * (t + t′)) + Ds[1]/(2 * lambda) * exp(-lambda * abs(t - t′))

In [ ]:
figure(figsize=(7, 3))

vmin = 0
vmax = Ds[1]/(2lambda)
ax = subplot(121) # plt.gca()
heatmap = ax.pcolormesh(t_scale * X, t_scale * Y, F.data, cmap=cmap, rasterized=true, vmin=vmin, vmax=vmax)
heatmap.set_edgecolor("face")
ax.set_aspect("equal")
cbar = colorbar(mappable=heatmap)
cbar.formatter.set_powerlimits((0, 0))
ax.set_xlabel("\$\\lambda t\$")
ax.set_ylabel("\$\\lambda t'\$")
ax.set_xlim(0, t_scale * T)
ax.set_ylim(0, t_scale * T)
ax.set_xticks(t_scale .* [0, T/2, T])
ax.set_yticks(t_scale .* [0, T/2, T])

ax = subplot(122)
heatmap = ax.pcolormesh(t_scale * X, t_scale * Y, (abs.(F.data .- F_ana.(X, Y))), cmap="gist_gray", rasterized=true)#, vmin=1e-9, vmax=1e-6)
heatmap.set_edgecolor("face")
ax.set_aspect("equal")
cbar = colorbar(mappable=heatmap)
cbar.formatter.set_powerlimits((0, 0))
ax.set_xlabel("\$\\lambda t\$")
# ax.set_ylabel("\$\\lambda t'\$")
ax.set_xlim(0, t_scale * T)
ax.set_ylim(0, t_scale * T)
ax.set_xticks(t_scale .* [0, T/2, T])
ax.set_yticks(t_scale .* [0, T/2, T])
ax.set_yticklabels([])

tight_layout(pad=0.75, w_pad=0.25, h_pad=0)
# savefig("brownian_motion_example_t_tp.pdf")

## Error scaling

In [ ]:
# epsilons = [2.0^(-k) for k in 6:19]
# err_data = [(0.0, 0.0, 0.0) for _ in 1:length(epsilons)]
# for (k, eps) in enumerate(epsilons)
#     print(k, ": ")
    
#     sol, _ = kbsolve(f_vert, f_diag, [F], tspan, dtini=1e-16, atol=eps*1e-2, rtol=eps)#, dtmax=eps)
# #     sol, _ = kbsolve(f_vert, f_diag, [F], tspan, dtini=1e-15, atol=1e-0, rtol=1e-0, dtmax=eps)
    
#     n = length(sol.t) - 1
#     print(n, ", ")
    
#     Y, X = np.meshgrid(sol.t[1:end], sol.t[1:end])
#     err_data[k] = (n, (1.0/n)^2 * norm(F_ana.(X, Y) - F.data, 1), eps)
# end

In [ ]:
# xdata = log10.([x[1] for x in err_data])
# ydata = log10.([x[2] for x in err_data]);

In [ ]:
# using LsqFit
# func_power = (n, p) -> 10^(p[2]) * n^(-p[1]);
# fit_func = (n, p) -> -p[1] .* n .+ p[2];
# fit_result = curve_fit(fit_func, xdata, ydata, [2.0, 1]);
# coef(fit_result)

In [ ]:
# figure(figsize=(7, 3))

# ax = subplot(121)
# plot(xdata, map(x -> fit_func(x, coef(fit_result)), xdata), "--k", lw=2,
#      label="\$\\mathcal{O}(h^{"*string(coef(fit_result)[1] |> x -> floor(x, sigdigits=4))*"})\$")
# plot(xdata, ydata, "s", lw=0, ms=5,
#      markerfacecolor="C0", markeredgewidth=0.25, markeredgecolor="k")
# xlim(0, 5)
# # ax.set_xticks([2.5, 2.75, 3.0, 3.25])
# xlabel("\$\\log(n)\$")
# ylabel("\$\\log(\\epsilon_{\\mathrm{abs}})\$")
# legend(loc="best", handlelength=1.8, frameon=false, borderpad=0, labelspacing=0)

# ax = subplot(122)
# plot(log10.([x[3] for x in err_data]), log10.([x[1] for x in err_data]), "s", ms=5,
#      markerfacecolor="C0", markeredgewidth=0.25, markeredgecolor="k")
# ax.yaxis.set_label_position("right")
# # xlabel("\$\\log(\\mathrm{dtmax})\$")
# xlabel("\$\\log(\\mathrm{rtol})\$")
# ylabel("\$\\log(n)\$", labelpad=16)

# tight_layout(pad=0.1, w_pad=0.75, h_pad=0)
# # savefig("error_scaling_brownian.pdf")

## Testing

In [ ]:
figure(figsize=(7, 3))

ax = subplot(121)
plot(sol.t, [F.data[k, k] for k in 1:n+1], "-", c="C0", label="\$xxx\$", lw=1.5)
plot(sol.t, F_ana.(sol.t, sol.t), c="C0", ls="--", lw=4.0, alpha=0.4)
xlim(0, lambda * T)
ylim(0, 1.1 * D/(2lambda))
yticks([0, 1, 2, 3, 4, 5])
xlabel("\$\\lambda t\$")
ylabel("\$F(t, t)\$")

ax = subplot(122)
plot(sol.t, abs.([F.data[k, k] for k in 1:n+1]  .- F_ana.(sol.t, sol.t)), ls="-", c="r", lw=1.5)
xlim(0, lambda * T)
ylim(0, )
# yticks([0, 1, 2, 3, 4, 5])
xlabel("\$\\lambda t\$")
ylabel("\$\\left|F(t, t) - \\mathcal{F}(t, t)\\right|\$", labelpad=16)
ax.yaxis.set_label_position("right")
# ax.legend(loc="best", handlelength=1.9, frameon=false, borderpad=0, labelspacing=0.25)
ticklabel_format(axis="y", style="sci", scilimits=(-0, 0))

tight_layout(pad=0.1, w_pad=1, h_pad=0)

# savefig("brownian_motion_example.pdf")

In [ ]:
figure(figsize=(7, 3))

ax = subplot(121)
plot(sol.t, [F.data[1, k] for k in 1:n+1], "-", c="C0", label="\$xxx\$", lw=1.5)
plot(sol.t, F_ana.(1, sol.t), c="C0", ls="--", lw=3.0, alpha=0.5)
xlim(0, lambda * T)
ylim(0, D/(2lambda))
# yticks([0, 1, 2, 3, 4, 5])
xlabel("\$\\lambda T\$")
ylabel("\$F(t, t)\$")

ax = subplot(122)
plot(sol.t, [F.data[k, k] for k in 1:n+1]  .- (F_ana.(sol.t, sol.t)), ls="-", c="r", lw=1.5)
xlim(0, lambda * T)
# ylim(0, 5.1)
# yticks([0, 1, 2, 3, 4, 5])
xlabel("\$\\lambda T\$")
ylabel("\$F(t, t) - \\mathcal{F}(t, t)\$", labelpad=16)
ax.yaxis.set_label_position("right")
# ax.legend(loc="best", handlelength=1.9, frameon=false, borderpad=0, labelspacing=0.25)
ticklabel_format(axis="y", style="sci", scilimits=(-0, 0))

tight_layout(pad=0.1, w_pad=1, h_pad=0)

# savefig("brownian_motion_example.pdf")

In [ ]:
figure(figsize=(7, 3))

vmin = 0
vmax = 1e-3
ax = plt.gca()
heatmap = ax.pcolormesh(t_scale * X, t_scale * Y, F_ana.(X, Y) - F.data, cmap=cmap, rasterized=true) # vmin=vmin, vmax=vmax
heatmap.set_edgecolor("face")
ax.set_aspect("equal")
cbar = colorbar(mappable=heatmap)
cbar.formatter.set_powerlimits((0, 0))
ax.set_xlabel("\$\\lambda t\$")
ax.set_ylabel("\$\\lambda t'\$")
ax.set_xlim(0, t_scale * T)
ax.set_ylim(0, t_scale * T)
ax.set_xticks(t_scale .* [0, T/2, T])
ax.set_yticks(t_scale .* [0, T/2, T])
tight_layout()

In [ ]:
figure(figsize=(5, 3))

imshow(F.data |> imag, ColorMap(cmap))
colorbar(orientation="vertical")
ax = gca()
# ax.set_xticks([0, n/2 + 1, n + 1])
# ax.set_xticklabels([0, lambda .*  T/2, abs(lambda) .* T])
ax.xaxis.set_ticks_position("top")
ax.xaxis.set_label_position("top")
# ax.set_yticks([0, n/2 + 1, n + 1])
# ax.set_yticklabels([0, lambda .* T/2, abs(lambda) .*  T])
xlabel("\$\\lambda t'\$")
ylabel("\$\\lambda t\$")
# clim((0, 5))
tight_layout()